<img style="float: left;" src="./images/logo.png" width="400"/>

This code is from the official GFED website

In [ ]:
months       = '01','02','03','04','05','06','07','08','09','10','11','12'
sources      = 'SAVA','BORF','TEMF','DEFO','PEAT','AGRI'

In this example we will calculate annual CO emissions for the 14 GFED basisregions over 1997-2014.
The distributed version of GFED4 stopped in 2024, so that we could not look at el pont de Vilomara here. [GFED5](https://essd.copernicus.org/articles/15/5227/2023/essd-15-5227-2023-discussion.html) is also available but stopped in 2020. 

Please adjust the code to calculate emissions for your own specie, region, and time period of interest. 

Please first get (from )the GFED4.1s files and the GFED4_Emission_Factors_Summary.csv to your computer and adjust the directory where you placed them below.
- data is available [here](https://www.dropbox.com/scl/fi/fgi1m3nnk436k9a0bhdpi/GFED4.1s.tar.gz?rlkey=vlaobhovce3v3lhr6uhfk4fd5&st=989ruqnp&dl=0)
- or if you are on andromeda the data are availabel here `/data/IMFSE/FBM_unit05/Emission/GFED4.1s`


In GFED4.1s biomass burning dry matter (DM) emissions are provided in kg DM/m2/month in the yearly file `GFED4.1s_xxxx.hdf5`. This corresponds to the mass of burnt fuel

## Read in emission factors

In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

In [ ]:
directory    = '/data/IMFSE/FBM_unit05/Emission/GFED4.1s/data/'

In [ ]:
species = [] # names of the different gas and aerosol species
EFs     = np.zeros((41, 6)) # 41 species, 6 sources

data_EF = pd.read_csv(directory+'/GFED4_Emission_Factors_Summary.csv', header=10)
EF_CO = np.array(data_EF.iloc[3])[1:]


# we are interested in CO for this example (4th row):

start_year = 1997
end_year   = 2014

## Make table with summed DM emissions for each region, year, and source

The code below load the burnt dry mass computed per month from the burn area product. It is available at global scale, monthly, and at 0.25 degree spatial resolution from mid-1995 through 2014. The data were derived by combining 500-m MODIS burned area maps with active fire data from the Tropical Rainfall Measuring Mission (TRMM) Visible and Infrared Scanner (VIRS) and the Along-Track Scanning Radiometer (ATSR) family of sensors. For additional information, refer to [Giglio et al., 2013](https://doi.org/10.1002/jgrg.20042).

In [ ]:

CO_table = np.zeros((15, end_year - start_year + 1)) # region, year
years =[]

for year in range(start_year, end_year+1):
    string = directory+'/GFED4.1s_'+str(year)+'.hdf5'
    f = h5py.File(string, 'r')

    years.append(year)
    
    if year == start_year: # these are time invariable    
        basis_regions = f['/ancill/basis_regions'][:]
        grid_area     = f['/ancill/grid_cell_area'][:]

        Regions=['BONA', 'TENA', 'CEAM', 'NHSA', 'SHSA', 'EURO', 'MIDE', 'NHAF' ,'SHAF', 'BOAS', 'TEAS', 'SEAS', 'EQAS', 'AUST', 'Global' ]
        #Boreal North America (BONA), 
        #Temperate North America (TENA), 
        #Central America (CEAM), 
        #Northern Hemisphere South America (NHSA), 
        #Southern Hemisphere South America (SHSA), 
        #Europe (EURO), 
        #Middle East (MIDE), 
        #Northern Hemisphere Africa (NHAF), 
        #Southern Hemisphere Africa (SHAF), 
        #Boreal Asia (BOAS), 
        #Central Asia (CEAS), 
        #Southeast Asia (SEAS), 
        #Equatorial Asia (EQAS) 
        #Australia and New Zealand (AUST).
    
    CO_emissions = np.zeros((720, 1440))
    for month in range(12):
        # read in DM emissions
        string = '/emissions/'+months[month]+'/DM'
        DM_emissions = f[string][:]
        for source in range(6): # 6 source of fire emission are: SAVA,BORF,TEMF,DEFO,PEAT,AGRI
            # read in the fractional contribution of each source
            string = '/emissions/'+months[month]+'/partitioning/DM_'+sources[source]
            contribution = f[string][:]
            # calculate CO emissions as the product of DM emissions (kg DM per 
            # m2 per month), the fraction the specific source contributes to 
            # this (unitless), and the emission factor (g CO per kg DM burned)
            CO_emissions += DM_emissions * contribution * float(EF_CO[source])
    
    
    # fill table with total values for the globe (row 15) or basisregion (1-14)
    for region in range(15):
        if region == 14:
            mask = np.ones((720, 1440))
        else:
            mask = basis_regions == (region + 1)            
    
        CO_table[region, year-start_year] = np.sum(grid_area * mask * CO_emissions)
        
    print(year)
        
# convert to Tg CO 
CO_table = CO_table / 1E12
#convert to dataframe
CO_df = pd.DataFrame(CO_table, index=Regions, columns=years)
print('yearly CO emission in Tg per regions area')
print(CO_df)


## To go further

1. on the 18 years, which region is the principal source of CO emitted from burning vegetation?
2. Can you compute the same for CO2? Is the spatial distribution per area the same?
3. Using the result from the last exercise of the introductionary python lecture (see [here](https://github.com/3dfirelab/python-intro-course/blob/master/06-pandasExcercise.ipynb)) can you calculate total carbone emission for you country? 